Importing libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json 
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

Importing the data

In [ ]:
with open('intents.json') as intents:
  data = json.load(intents)

# creating lists 
tags=[]
patterns=[]
responses={}
for intent in data['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    patterns.append(lines)
    tags.append(intent['tag'])

# creating a dataframe
data1 = pd.DataFrame({"patterns":patterns,
                     "tags":tags})
print(data1)

                                      patterns      tags
0                                           Hi  greeting
1                                          Hey  greeting
2                             Is anyone there?  greeting
3                                        Hello  greeting
4                                    Greetings  greeting
5                 Can i book for a test drive?  services
6    Any information about warranty on the car  services
7   Which after sales services do you provide?  services
8                                               services
9                What latest make do you have?      make
10       Which 4 wheel drive make is the best?      make
11                     Do you have hatchbacks?      make
12                  Do you have electric cars?      make
13                                        Audi      make
14                                    Mercedes      make
15                                      Toyota      make
16                             

Data pre-processing i.e tokenization where each word gets a unique token.

In [ ]:
# My intents are punctuated so lets remove them.
import string
data1['patterns'] = data1['patterns'].apply(lambda word:[letters.lower() 
for letters in word if letters not in string.punctuation])
data1['patterns']=data1['patterns'].apply(lambda wrd:''.join(wrd))

# tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(data1['patterns'])
train = tokenizer.texts_to_sequences(data1['patterns'])

# padding to obtain a single tensor since the patterns to be tokenized have different lengths
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

# encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data1['tags'])

In [ ]:
# setting the input length, output length and vocabulary
# for input length
input_shape= x_train.shape[1]
print(input_shape)

8


In [ ]:
# defining the vocabulary
vocabulary = len(tokenizer.word_index)
print("Number of unique words : ",vocabulary)

Number of unique words :  67


In [ ]:
# for the output length
output_length= le.classes_.shape[0]
print('Output length : ', output_length)

Output length :  6


Model

In [ ]:
# creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)

# compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer = 'adam',metrics=['accuracy'])

# training the model 
train = model.fit(x_train,y_train,epochs=500)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 1.7913 - accuracy: 0.1333
Epoch 2/500
1/1 [==============================] - 0s 16ms/step - loss: 1.7890 - accuracy: 0.1667
Epoch 3/500
1/1 [==============================] - 0s 13ms/step - loss: 1.7868 - accuracy: 0.2333
Epoch 4/500
1/1 [==============================] - 0s 17ms/step - loss: 1.7845 - accuracy: 0.3667
Epoch 5/500
1/1 [==============================] - 0s 17ms/step - loss: 1.7823 - accuracy: 0.3667
Epoch 6/500
1/1 [==============================] - 0s 18ms/step - loss: 1.7800 - accuracy: 0.3667
Epoch 7/500
1/1 [==============================] - 0s 14ms/step - loss: 1.7777 - accuracy: 0.3667
Epoch 8/500
1/1 [==============================] - 0s 16ms/step - loss: 1.7754 - accuracy: 0.3333
Epoch 9/500
1/1 [==============================] - 0s 15ms/step - loss: 1.7730 - accuracy: 0.3333
Epoch 10/500
1/1 [==============================] - 0s 16ms/step - loss: 1.7706 - accuracy: 0.3333
Epoch 11/500
1/1 [===

Testing 

In [ ]:
import random

while True:
  texts_p =[]
  prediction_pattern = input('You : ')

  # removing punctuation and coverting to lowercase
  prediction_pattern = [letters.lower() for letters in prediction_pattern if letters not in string.punctuation]
  prediction_pattern = ''.join(prediction_pattern)
  texts_p.append(prediction_pattern)

  # tokenizing and padding
  prediction_pattern = tokenizer.texts_to_sequences(texts_p)
  prediction_pattern = np.array(prediction_pattern).reshape(-1)
  prediction_pattern = pad_sequences([prediction_pattern],input_shape)

  # output of the model
  output = model.predict(prediction_pattern)
  output = output.argmax()

  # finding the correct tag and predicting it 
  response_tag =le.inverse_transform([output])[0]
  print("Carbot : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : hi
Carbot :  Hello, how may i help you?
You : bye
Carbot :  Have a great day ahead!
